# Metabarcoding data analysis pipeline
The following pipeline was used to analyze metabarcoding data from water and sediment samples collected in Indonesia in the INSERT PUB INFO HERE. The sequence data generated in this study were amplicons of the V9 hypervariable region of 18S rDNA, which is about 130bp in length and were sequenced on the Illumina MiSeq using paired end sequencing. 

For more detailed information on our study and methods please see the following publication (link publication).

## Quality assessment
Quality assessment of raw reads initially done using FastQC (Andrew, 2010).


## Sequence trimming
Forward and reverse primer sequences were removed from the front and back ends of forward and reverse reads using Cutadapt (Martin, 2011). The first command removes the forward and reverse primer sequences from the front end of the forward and reverse reads respectively. Untrimmed outputs were saved to later evaluate potential error in the trimming process.

In [ ]:
cutadapt -g TTGTACACACCGCCC \
         -G CCTTCYGCAGGTTCACCTAC \
         -q 20 \
         --untrimmed-output untrimmed_EB${SLURM_ARRAY_TASK_ID}_R1.fastq \
         --untrimmed-paired-output untrimmed_EB${SLURM_ARRAY_TASK_ID}_R2.fastq \
         -o EB${SLURM_ARRAY_TASK_ID}_cox1_R1.fastq \
         -p EB${SLURM_ARRAY_TASK_ID}_cox1_R2.fastq \
         EB${SLURM_ARRAY_TASK_ID}_R1.fastq \
         EB${SLURM_ARRAY_TASK_ID}_R2.fastq

The second command removed the reverse complements of the forward and reverse primer sequences from the back ends of the reverse and forward reads respectively. This step also removes any sequences under 100bp and as above saves the untrimmed output to evaluate potential error in the trimming process.

In [ ]:
cutadapt -a GTAGGTGAACCTGCRGAAGG \
         -A GGGCGGTGTGTACAA \
         -q 20 \
         --untrimmed-output un_untrimmed_EB${SLURM_ARRAY_TASK_ID}_R1.fastq \
         --untrimmed-paired-output un_untrimmed_EB${SLURM_ARRAY_TASK_ID}_R2.fastq \
         -o EB${SLURM_ARRAY_TASK_ID}_R1_clean.fastq \
         -p EB${SLURM_ARRAY_TASK_ID}_R2_clean.fastq \
         EB${SLURM_ARRAY_TASK_ID}_cox1_R1.fastq 
         EB${SLURM_ARRAY_TASK_ID}_cox1_R2.fastq \
         --pair-filter=any \
         --minimum-length 100 \
         --too-short-output too_short_EB${SLURM_ARRAY_TASK_ID}_R1.fastq \
         --too-short-paired-output too_short_EB${SLURM_ARRAY_TASK_ID}_R2.fastq

The final step carried out in the trimming process was to concatenate the untrimmed output and trimmed output from the second step above. Due to the sequencing run lengths, not all sequences had the full reverse complement sequnce at the end of the read and therefore were thrown into untrimmed output despite being high quality and correct length. The concatenation step allowed me to retain as many reads as possible while also making sure those with reverse complement squences were properly trimmed.

In [ ]:
cat CleanReads/EB${SLURM_ARRAY_TASK_ID}_R1_clean.fastq un_untrimmed/un_untrimmed_EB${SLURM_ARRAY_TASK_ID}_R1.fastq > EB${SLURM_ARRAY_TASK_ID}_R1_cat.fastq

cat CleanReads/EB${SLURM_ARRAY_TASK_ID}_R2_clean.fastq un_untrimmed/un_untrimmed_EB${SLURM_ARRAY_TASK_ID}_R2.fastq > EB${SLURM_ARRAY_TASK_ID}_R2_cat.fastq

## Importing reads into QIIME2
The remainder of this data analysis was carried out in QIIME2 (2020.6) (Bolyen, et al. 2019) using various plugins available within the program. Since trimming was done outside of QIIME2, sequence files were imported using a manifest file, an example of which is provided in the github repository.

The first command, activates the QIIME2 environment, while the second command imports the sequence data as a QIIME2 artifcat ('.qza' file).

In [ ]:
conda activate qiime2-2020.6

In [ ]:
qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path ../IndoV9_022818_manifest.csv \
--output-path paired-end-demux.qza \
--input-format PairedEndFastqManifestPhred33

Once imported the summarize command was used to generate an interactive quality plot of the imported sequences as a QIIME2 visualiztion ('.qzv' file).

In [ ]:
qiime feature-table summarize \
--i-table paired-end-demux.qza \
--o-visualization paired-end-demux.qzv \
--m-sample-metadata-file Master_V9_MappingFile.txt

## Denoising and merging
Paired sequences were denoised and merged using the DADA2 plugin within QIIME2 (Callahan, et al. 2013). In this step, sequencing errors were detected and corrected where possible, filtered, and merged. Following merging, the merged sequences were checked for chimeras and chimeras were then filtered out.

Truncation lengths for forward and reverse reads were chosen using the interactive quality plot generated in the previous step. Reads were truncated at the position where quality began to drop off, while maintaining at least 30bp of overlap between forward and reverse reads for merging.

The final output of this step is a list of amplicon sequence variants (ASVs) (`rep-seqs.qza`), an ASV table (how many of each ASV were in each sample)(`table.qza`), and a statistics summary showing how many reads were lost at each step (`stats.qza`). 

In [ ]:
qiime dada2 denoise-paired \
--i-demultiplexed-seqs paired-end-demux.qza \
--p-trunc-len-f 100 \
--p-trunc-len-r 100 \
--o-table table.qza \
--o-representative-sequences rep-seqs.qza \
--o-denoising-stats stats.qza

To evaluate the denoising/merging statistics and view information from the ASV table and list, we generated visualizations (`.qzv` files) for each of the `.qza` files generated above using the `metadata tabulate`,`summarize`, and `tabulate-seqs` commands.

In [ ]:
qiime metadata tabulate \
--m-input-file stats.qza \
--o-visualization stats.qzv

In [ ]:
qiime feature-table summarize \
--i-table table.qza \
--o-visualization table.qzv \
--m-sample-metadata-file Master_V9_MappingFile.txt

In [ ]:
qiime feature-table tabulate-seqs \
--i-data rep-seqs.qza \
--o-visualization rep-seqs.qzv

To view these and any other visualizations generated in this script we can use `qiime tools view`.

In [ ]:
qiime tools view stats.qzv

In [ ]:
qiime tools view table.qzv

In [ ]:
qiime tools view rep-seqs.qzv

*OTU Clustering*

Diversity statistics and other analyses were run first using ASVs and later on operational taxonomic units (OTUs) to compare outputs and choose the one that best represented our data and study goals. Below I include the command used to cluster our ASVs into OTUs at 97% and 99% similarity, but the rest of the script provided in this notebook will continue with the ASV data.

In [ ]:
qiime vsearch cluster-features-de-novo \
  --i-table table.qza \
  --i-sequences rep-seqs.qza \
  --p-perc-identity 0.99 \
  --o-clustered-table otu-table-99.qza \
  --o-clustered-sequences otus-99.qza

In [ ]:
qiime vsearch cluster-features-de-novo \
  --i-table table.qza \
  --i-sequences rep-seqs.qza \
  --p-perc-identity 0.97 \
  --o-clustered-table otu-table-97.qza \
  --o-clustered-sequences otus-97.qza

Ultimately, our statistics showed the same results between ASV and OTU data. Though the scripts proceed forward with ASV data, the OTU results can easily be obtained using the same commands and substituting in the OTU table (`otu-table-99.qza` or `otu-table-97.qza`) and OTU list (`otus-99.qza` or `otus-97.qza`) for the ASV table (`table.qza`) and ASV list (`rep-seqs.qza`).

## Align ASVs and construct tree
In this step I used MAFFT to align resulting ASVs and construct and then root a tree. This is all done in the one command provided below.

In the case where I was combining samples from multilpe sequencing runs an additional step was done prior to this to combine `rep-seqs.qza` and `table.qza` files from individual sequencing runs into a single combined `rep-seqs.qza` and `table.qza`.

The output from this step will be used later for diversity statistics and other analyses.

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
--i-sequences rep-seqs.qza \
--o-alignment aligned-rep-seqs.qza \
--o-masked-alignment masked-aligned-rep-seqs.qza \
--o-tree unrooted-tree.qza \
--o-rooted-tree rooted-tree.qza

## Building and training taxonomic classifier
The next step was to construct and train a feature classifier to my target region (18S V9) using a reference database. I used the Protist Ribosomal Reference (PR2) Database v4.12.0 (https://pr2-database.org/).

The first two commands import the reference sequences (`ref-seqs.qza`) and reference taxonomy (`ref-taxonomy.qza`) into QIIME2 respectively.

In [ ]:
qiime tools import \
--type 'FeatureData[Sequence]' \
--input-path /users/erinborbee/Desktop/DataAnalysis/pr2-classifier/pr2_version_4.12.0_18S_mothur.fasta \
--output-path pr2-seqs.qza

In [ ]:
qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-format HeaderlessTSVTaxonomyFormat \
--input-path  /users/erinborbee/Desktop/DataAnalysis/pr2-classifier/pr2_version_4.12.0_18S_mothur.tax \
--output-path pr2-taxonomy.qza

Once reference sequences and taxonomy are imported, the next step was to extract our target region (18S V9) from the reference sequences (`ref-seqs.qza`) using our primer sequences. We did this because studies from other amplicon regions have shown to have improved taxonomic classification when using a trained classifier.

In [ ]:
qiime feature-classifier extract-reads \
--i-sequences pr2-seqs.qza \
--p-f-primer TTGTACACACCGCCC \
--p-r-primer CCTTCYGCAGGTTCACCTAC \
--o-reads ref-seqs.qza

The final step in this process was to train the actual classifier, which was done using the command below.

In [ ]:
qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads ref-seqs.qza \
--i-reference-taxonomy pr2-taxonomy.qza \
--o-classifier pr2-classifier.qza

## Taxonomic assignment
Once the classifier was trained, we then assigned taxonomy to our ASVs and visualized community composition using the taxa barplot command in QIIME2.

The first command provided assigns taxonomy to our ASVs and the second and third commands create visualizations of the taxonomy table (`taxonomy.qzv`) and community barplot respectively (`taxa-barplot.qzv`).

In [ ]:
qiime feature-classifier classify-sklearn \
--i-classifier pr2-classifier.qza \
--i-reads rep-seqs.qza \
--o-classification taxonomy.qza

In [ ]:
qiime metadata tabulate \
--m-input-file taxonomy.qza \
--o-visualization taxonomy.qzv

In [ ]:
qiime taxa barplot \
--i-table table.qza \
--i-taxonomy taxonomy.qza \
--m-metadata-file Master_V9_MappingFile.txt \
--o-visualization taxa-bar-plots.qzv

In [ ]:
qiime tools view taxa-bar-plots.qzv

### Filtering to protist taxa
Following taxonomic assignment the ASV table and ASV list were filtered to only sequences belonging to the Stramenopiles, Alveolates, and Rhizaria. While these are not the only protist lineages, they encompass the most abundant and diversity microbial eukaryotic groups as established by other metabarcoding surveys conducted across the world's oceans.

In [ ]:
qiime taxa filter-table \
--i-table table.qza \
--i-taxonomy taxonomy.qza \
--p-include Stramenopiles,Alveolata,Rhizaria \
--o-filtered-table SAR_table.qza

Once the table was generated, we used the summarize command to generate a visualization (`.qzv`). We then used this visualization to find the sampling depth needed for the core diversity metrics command.

In [ ]:
qiime feature-table summarize \
--i-table SAR_table.qza \
--o-visualization SAR_table.qzv \
--m-sample-metadata-file Master_V9_MappingFile.txt

## Diversity statistics
Diversity statististics were calculated in QIIME2 using the `core-metrics-phylogenetic` command, followed by individual commands for testing significance of different diversity metrics across individual metadata variables.

The `core-metrics-phylogenetic` command generates a directory with various alpha and beta diversity metrics including Shannon, richness, faith, and eveness for alpha diversity and Bray-Curtis, Jaccard, and weighted and unweighted UniFrac for Beta diversity. The sampling depth was chosen by using the `table.qzv` file generated above and using the minimum frequency of sequences across all of the samples.

In [ ]:
qiime tools view SAR_table.qzv

In [ ]:
 qiime diversity core-metrics-phylogenetic \
 --i-phylogeny rooted-tree.qza \
 --i-table SAR_table.qza \
 --p-sampling-depth [minimumFrequency] \
 --m-metadata-file Master_V9_MappingFile.txt \
 --output-dir core-metrics-results

### Rarefaction
Rarefaction curves were plotted to evaluate sampling depth and see if sampling reached saturation. The same sampling depth mentioned in the above command was used here.

In [ ]:
qiime diversity alpha-rarefaction \
--i-table SAR_table.qza \
--i-phylogeny rooted-tree.qza \
--p-max-depth [minimumFrequency] \
--m-metadata-file Master_V9_MappingFile.txt \
--o-visualization alpha-rarefaction.qzv

### Alpha Diversity
The variables we were interested in testing significance for were categorical variables. Therefore, Kruskal-Wallis was used to test for significance of these variables for alpha diversity metrics (Shannon diversity & richness). The categorical variables for our samples are contained in the `Master_V9_MappingFile.txt`. The first command generates a visualization for the significance of Shannon diversity by metadata groups and the second command generates a visualization for the significance of ASV richness by metadata groups.

In [ ]:
qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics-results/shannon_vector.qza \
--m-metadata-file Master_V9_MappingFile.txt \
--o-visualization core-metrics-results/shannon-group-significance.qzv

In [ ]:
qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics-results/observed_features_vector.qza \
--m-metadata-file Master_V9_MappingFile.txt \
--o-visualization core-metrics-results/richness-group-significance.qzv

### Beta Diversity
Since the variables we were interested in again were categorical variables, ANOSIM and PERMANOVA were used to test for significance in community composition shifts across different metadata groups. Ultimately we chose to stick with ANOSIM because it was the more conservative approach given our experimental design. The following commands create visualizations for the ANOSIM statistics for differences in beta diversity across sampling regions, protected vs unprotected areas, and different fisheries management regimes in that order.

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix core-metrics-results/bray_curtis_distance_matrix.qza \
--m-metadata-file Master_V9_MappingFile.txt \
--m-metadata-column region \
--o-visualization core-metrics-results/bray_curtis-region-significance.qzv \
--p-pairwise --p-method anosim

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix core-metrics-results/bray_curtis_distance_matrix.qza \
--m-metadata-file Master_V9_MappingFile.txt \
--m-metadata-column MPA \
--o-visualization core-metrics-results/bray_curtis-MPA-significance.qzv \
--p-pairwise --p-method anosim

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix core-metrics-results/bray_curtis_distance_matrix.qza \
--m-metadata-file Master_V9_MappingFile.txt \
--m-metadata-column restriction \
--o-visualization core-metrics-results/bray_curtis-restriction-significance.qzv \
--p-pairwise --p-method anosim

In order to visualize beta diversity results, an emperor plot was constructed using the PCoA vector generated in the `core-metrics-phylogenetic` command. This command produces an interactive, 3-dimensional ordination that allows you to change color and shapes of points by any metadata variable.

In [ ]:
qiime emperor plot \
--i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \
--m-metadata-file  Master_V9_MappingFile.txt  \
--o-visualization core-metrics-results/bray-curtis-emperor.qzv